In [46]:
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

## Scrape Data from URL using BeautifulSoup Library

In [47]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
Source = BeautifulSoup(website_url,'lxml')
Table = Source.find('table',{'class':'wikitable sortable'})
headers = "Postal Code, Borough, Neighbourhood"

table1=""
for tr in Table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
    
file = open("toronto.csv", "wb")
file.write(bytes(table1,encoding="ascii", errors = "ignore"))

8738

## Create table

In [48]:
df = pd.read_csv('toronto.csv', header = None)
df.columns = ["Postal Code", "Borough", "Neighbourhood"]

## Remove rows where borough are unassigned

In [49]:
UnassignedNames = df[ df['Borough'] == 'Not assigned'].index
df.drop(UnassignedNames , inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Replace rows where neighbourhood is not assigned with borough

In [50]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Combine neighbourhoods with the same postal code

In [51]:
CombinedPostalCode = df.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df_1 = CombinedPostalCode.reset_index()
df_1.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


## Print Shape

In [52]:
df_1.shape

(103, 3)